In [1]:
import json
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
labels = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [3]:
with open('data.json', 'r') as f:
    data = json.load(f)

In [4]:
unique_colors = set()
for circle in data:
    unique_colors.add(circle['color'])
    
unique_x = set()
for circle in data:
    unique_x.add(int((circle['x'] - 3)/11))
sorted(unique_x)

unique_y = set()
for circle in data:
    unique_y.add(int((circle['y'] - 3)/8))
sorted(unique_y)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [9]:
print(unique_colors)
print(unique_x)
print(unique_y)

{'#49B882', '#98CFAC', '#D0B8FB', '#FAD9E7', '#99CEFF', '#FCC89B', '#E365A6', '#F9FAFB'}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18}


In [6]:
Counter(circle['color'] for circle in data)

Counter({'#49B882': 503,
         '#98CFAC': 11,
         '#99CEFF': 6,
         '#D0B8FB': 1,
         '#E365A6': 13,
         '#F9FAFB': 26,
         '#FAD9E7': 6,
         '#FCC89B': 1})

In [7]:
Counter(circle['x'] for circle in data)

Counter({3: 9,
         14: 6,
         25: 4,
         36: 6,
         47: 5,
         58: 8,
         69: 6,
         80: 5,
         91: 8,
         102: 8,
         113: 4,
         124: 6,
         135: 8,
         146: 6,
         157: 8,
         168: 8,
         179: 4,
         190: 4,
         201: 8,
         212: 5,
         223: 6,
         234: 7,
         245: 7,
         256: 5,
         267: 8,
         278: 4,
         289: 8,
         300: 4,
         311: 5,
         322: 8,
         333: 5,
         344: 6,
         355: 4,
         366: 6,
         377: 8,
         388: 7,
         399: 5,
         410: 7,
         421: 4,
         432: 6,
         443: 8,
         454: 4,
         465: 9,
         476: 6,
         487: 2,
         498: 7,
         509: 5,
         520: 7,
         531: 8,
         542: 8,
         553: 6,
         564: 5,
         575: 3,
         586: 8,
         597: 4,
         608: 8,
         619: 3,
         630: 8,
         641: 4,
       

In [8]:
Counter(circle['y'] for circle in data)

Counter({3: 2,
         11: 6,
         19: 8,
         27: 11,
         35: 13,
         43: 18,
         51: 24,
         59: 29,
         67: 28,
         75: 37,
         83: 36,
         91: 45,
         99: 41,
         107: 44,
         115: 42,
         123: 46,
         131: 46,
         139: 46,
         147: 45})